In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from airfoil._WingSegment import WingSegment
from airfoil._Decomposer import Decomposer
from airfoil.wing import (
    calculated_wing_cube_loading,
    angle_degrees_to_slope,
    auto_piecewise,
    mirror
)
from airfoil import Airfoil
from itertools import pairwise
import pyvista as pv

In [ ]:
aspect_ratio = 12
root_chord = 130

wing_span = aspect_ratio * root_chord
half_span = wing_span/2

sweep = 4
dihedral = 1
compound_dihedral = 30

body_width = 50


sample_locations = np.array([
    0,
    body_width/2,
    body_width/2 + 100,
    body_width/2 + 200,
    body_width/2 + 300,
    body_width/2 + 400,
    compound_break_location := body_width/2 + 500,
    half_span,
])

In [ ]:
def create_sampler(samples):
    return lambda x: np.interp(np.abs(x), *samples.T)

samples_leading_edge = np.array([
    [0            , 0                                          ],
    [body_width/2 , 0                                          ],
    [half_span    , -(half_span-body_width/2) * angle_degrees_to_slope(sweep)],
])
sample_leading_edge = create_sampler(samples_leading_edge)

samples_chord = np.array([
    [0                           , 1.00],
    [body_width/2                , 1.00],
    [body_width/2 + 200          , 0.85],
    [compound_break_location     , 0.80],
    [half_span                   , 0.80],
]) * np.array([[1, root_chord]])
sample_chord = create_sampler(samples_chord)

sample_trailing_edge = lambda x_b: sample_leading_edge(x_b) - sample_chord(x_b)

samples_washout = np.array([
    [0                              , 3],
    #[body_width/2                   , 3],
    [body_width/2 + 100             , 3],
    [compound_break_location        , 0],
    [half_span                      , 0]
])
sample_washout = create_sampler(samples_washout)

samples_thickness = np.array([
    [0                      , 0.20],
    [body_width/2           , 0.20],
    [body_width/2 + 100     , 0.13],
    [compound_break_location, 0.10],
    [half_span              , 0.10],
])
sample_thickness = create_sampler(samples_thickness)

sample_dihedral= mirror(auto_piecewise([
    (compound_break_location, lambda x: x*angle_degrees_to_slope(dihedral)),
    (half_span              , lambda x: x*angle_degrees_to_slope(compound_dihedral)),
]))
sample_dihedral_angle = lambda x_b: np.rad2deg(np.atan2(sample_dihedral(x_b+0.001)-sample_dihedral(x_b-0.001), 0.001*2))



fig, axs = plt.subplots(4,sharex=True, figsize=(15,8))
(ax1,ax2, ax3, ax4) = axs
x_b = np.linspace(-half_span,half_span,200)
ax1.plot(x_b, sample_leading_edge(x_b), label="leading edge")
ax1.plot(x_b, sample_trailing_edge(x_b), label="trailing_edge")
ax1.set_aspect("equal")
ax2.plot(x_b, sample_dihedral(x_b), label = "dihedral")
ax2.set_aspect("equal")
ax3.plot(x_b, sample_washout(x_b), label="washout")
ax4.plot(x_b, sample_thickness(x_b), label="thickness")
for ax in axs:
    ax.legend()
    for sl in sample_locations:
        ax.axvline(sl,c="red",linestyle=":")

In [ ]:
mean_chord = np.sum(
    (
     (sample_chord(x_b))[1:]
    +(sample_chord(x_b))[:-1]
    )
    / 2
    * np.diff(x_b)
) / wing_span
mean_chord

In [ ]:
def create_foil(x_b, dihedral_deg:float=0):
    max_camber_position = 0.2
    chord = sample_chord(x_b)
    return (
        Airfoil.from_naca5(
            "standard",
            design_lift_coefficient=0.3,
            max_camber_position=max_camber_position,
            max_thickness=sample_thickness(x_b),
            chord_length=chord
        )
        .with_scale((1,1/np.cos(np.deg2rad(dihedral_deg))))
        .with_translation(np.array([-chord * max_camber_position,0]))
        .with_rotation(
            sample_washout(x_b)
        )
        .with_translation(np.array([ chord * max_camber_position,0]))
        .with_translation(np.array([
            -sample_leading_edge(x_b),
            sample_dihedral(x_b)
        ]))
    )
fig,ax = plt.subplots()

for x in sample_locations:
    create_foil(x,0).plot(ax=ax)

In [ ]:
segments  = [
    WingSegment(
        create_foil(
            sla, 
            dihedral_deg=sample_dihedral_angle(mid:=(slb+sla)/2),
        ),
        create_foil(
            slb, 
            dihedral_deg=sample_dihedral_angle(mid)
        ),
        slb-sla
    ) for sla, slb in pairwise(sample_locations)
]

In [ ]:
pt = pv.Plotter()
o = 0
volume = 0
wing_meshes = []
for segment in segments:
    o += segment.length/2
    msh = segment.to_mesh()
    volume+=msh.volume*2
    wing_meshes.append(msh.translate([o,0,0]))
    wing_meshes.append(msh.scale([-1,1,1]).translate([-o,0,0]).flip_faces())
    o += segment.length/2
#pt.enable_eye_dome_lighting()
wing_mesh = wing_meshes[0]
for m in wing_meshes[1:]:
    wing_mesh+=m
pt.add_mesh(wing_mesh.rotate_x(-4).translate((0,0,60)))
pt.add_mesh(pv.Cylinder((0,500,0),direction=(0,1,0),radius=8,height=1000).translate((0,-180,0)))
pt.add_mesh
pt.show()

In [ ]:
# weight estimate
xps_foam_density_kg_per_meter_cubed = 1.3/(1.2*0.600*0.050)
wing_mass = volume/1000**3 * xps_foam_density_kg_per_meter_cubed*1000
mass_estimate = np.array([
    150, # electonics
    60,  # rod
    100, # tail
    wing_mass,
]).sum()
mass_estimate

In [ ]:
calculated_wing_cube_loading(
    weight = mass_estimate/1000, # kg
    area   = (wing_span*mean_chord)/1000**2
)

In [ ]:
pt.export_gltf("./data/outputs/2025 06 15 glider_wing_design.gltf")